In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

In [2]:
model = Sequential()
embedding_layer = Embedding(input_dim=10,output_dim=4,input_length=2)
model.add(embedding_layer)
model.compile('adam','mse')

In [3]:
input_data = np.array([[1,2]])
pred = model.predict(input_data)
print(input_data.shape)
print(pred)

1/1 [==============================] - 0s 93ms/step
(1, 2)
[[[ 0.044051    0.04555774  0.03470489 -0.03525679]
  [ 0.02662767  0.00593636 -0.00188937  0.0365881 ]]]


# first [ for array second one for aggregate two vectors
# [[
  # The vector For word 1 : [ 0.044051    0.04555774  0.03470489 -0.03525679]
 # The vector For word 2 : [ 0.02662767  0.00593636 -0.00188937  0.0365881 ]
 # first [ for array second one for aggregate two vectors ]]
 # As we see the length of output is 4.

In [9]:


# or access the embedding layer through the constructed model
# first `0` refers to the position of embedding layer in the `model`
# embeddings = model.layers[0].get_weights()[0]
embeddings = model.layers[0].get_weights()
embeddings

[array([[-0.01971984, -0.04984335,  0.02057571, -0.03237033],
        [ 0.044051  ,  0.04555774,  0.03470489, -0.03525679],
        [ 0.02662767,  0.00593636, -0.00188937,  0.0365881 ],
        [ 0.00802473, -0.00595004, -0.02466425,  0.01850195],
        [ 0.04209392, -0.00069617, -0.04870185,  0.02699608],
        [-0.03689731, -0.03607271, -0.04325372, -0.00563365],
        [ 0.03870176,  0.04522774,  0.03089739, -0.01539282],
        [-0.00060452, -0.00548015,  0.04173461,  0.01630619],
        [ 0.04565387,  0.01521132, -0.02844551, -0.01636089],
        [-0.02075909,  0.03396681, -0.01516876, -0.02283299]],
       dtype=float32)]

# The output above is refer two 10 wocabulary size that we set and each of row is represent the vector of words. that is important that the weights are above are initialize randomly and every time you run the cell get different numbers.
# for example :
# The first word (0) is represented by first row in this table, which is :
# [-0.04333381, -0.02326865, -0.00812379,  0.02167496]

# An example above has a problem because we don't train the embedding layer but in real text dataset we shold have done this. therefor in other eample we should see how train embedding layer.


1.   Tokenize the sentences into words.
2.   Create one-hot encoded vector for each word.
1.   Use padding to ensure all sequences are of same length.
2.   Pass the padded sequences as input to embedding layer.
1.   Flatten and apply Dense layer to predict the label.:





In [10]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

In [11]:
# Define 10 restaurant reviews
reviews =[
          'Never coming back!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome',
          'awesome services!',
          'rocks',
          'poor work',
          'couldn\'t have done better'
]
#Define labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [13]:
reviews[0]

'Never coming back!'

In [28]:
Vocab_size = 50
encoded_reviews = [one_hot(d,Vocab_size) for d in reviews]
print(f'encoded reviews: {encoded_reviews}')

encoded reviews: [[15, 25, 11], [44, 15], [7, 36], [10, 33], [44, 33], [48], [48, 48], [4], [15, 21], [42, 12, 7, 40]]


In [29]:
one_hot(reviews[0],Vocab_size)

[15, 25, 11]

# Difference between post and pre is in post use 0 at the end but in pre use 0 at the begining.

In [30]:
max_length = 4
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')
print(padded_reviews)

[[15 25 11  0]
 [44 15  0  0]
 [ 7 36  0  0]
 [10 33  0  0]
 [44 33  0  0]
 [48  0  0  0]
 [48 48  0  0]
 [ 4  0  0  0]
 [15 21  0  0]
 [42 12  7 40]]


In [31]:
max_length = 4
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='pre')
print(padded_reviews)

[[ 0 15 25 11]
 [ 0  0 44 15]
 [ 0  0  7 36]
 [ 0  0 10 33]
 [ 0  0 44 33]
 [ 0  0  0 48]
 [ 0  0 48 48]
 [ 0  0  0  4]
 [ 0  0 15 21]
 [42 12  7 40]]


# After padding it is time to pass input to the embedding layer.
# We will fix the length of embedded vectors for each word as 8 and the input length will be the maximum length which we have already defined as 4.

In [32]:
model = Sequential()
embedding_layer = Embedding(input_dim=Vocab_size,output_dim=8,input_length=max_length)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 433 (1.69 KB)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# Train the model

In [34]:
model.fit(padded_reviews,labels,epochs=100,verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 11ms/step - loss: 0.4529 - acc: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 0.4501 - acc: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 13ms/step - loss: 0.4473 - acc: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.4445 - acc: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4417 - acc: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4389 - acc: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4361 - acc: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4333 - acc: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4305 - acc: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4277 - acc: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 10ms/step - los

# Lets check the shape of the weight matrix.

In [35]:
print(embedding_layer.get_weights()[0].shape)

(50, 8)


In [36]:
print(embedding_layer.get_weights())

[array([[-2.31203958e-01,  1.86820880e-01,  1.68216407e-01,
        -2.04175085e-01,  1.68387517e-01, -2.67770767e-01,
        -1.87188357e-01, -2.19602570e-01],
       [ 2.65760086e-02,  2.87699699e-03, -3.34760435e-02,
         4.25954908e-03,  3.84280719e-02, -4.34306040e-02,
        -1.71516165e-02,  4.92638014e-02],
       [-2.32205037e-02,  4.60121073e-02,  3.79404165e-02,
         2.62589380e-03, -4.00102623e-02,  7.34820217e-03,
         2.63142847e-02,  3.72612812e-02],
       [ 2.03385614e-02,  1.11434348e-02,  1.05656311e-03,
        -7.76886940e-04, -2.26223823e-02,  1.85558088e-02,
        -4.89340685e-02, -7.93641806e-03],
       [ 1.78463817e-01, -1.58793807e-01, -1.91601187e-01,
         1.45624533e-01,  1.92026883e-01, -1.69741556e-01,
        -1.88673213e-01, -1.89551279e-01],
       [-5.94462082e-03, -5.18176705e-03, -4.69034687e-02,
        -1.27081163e-02,  3.43217291e-02,  1.17473714e-02,
        -3.95879634e-02,  3.30544375e-02],
       [-1.01365559e-02,  9.39179

#  first word, we get the following vector. is :
# [-2.31203958e-01,  1.86820880e-01,  1.68216407e-01,
  #      -2.04175085e-01,  1.68387517e-01, -2.67770767e-01,
   #     -1.87188357e-01, -2.19602570e-01]

# text_to_word_sequence

In [20]:

from keras.preprocessing.text import text_to_word_sequence
# define the document
text = 'The quick brown fox jumped over the lazy dog.'
# tokenize the document
result = text_to_word_sequence(text)
print(result)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']


In [23]:
# define the document
text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of the vocabulary
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
print(words)

8
{'jumped', 'the', 'fox', 'lazy', 'dog', 'brown', 'quick', 'over'}


# Converts a text to a sequence of indexes in a fixed-size hashing space.
# A list of integer word indices (unicity non-guaranteed).
#0 is a reserved index that won't be assigned to any word. Two or more words may be assigned to the same index, due to possible collisions by the hashing function. The probability of a collision is in relation to the dimension of the hashing space and the number of distinct objects.

In [22]:
'''
tf.keras.preprocessing.text.hashing_trick(
    text,
    n,
    hash_function=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    analyzer=None
)
'''

NameError: ignored

# One-hot encodes a text into a list of word indexes of size n
# This function receives as input a string of text and returns a list of encoded integers each corresponding to a word (or token) in the given input string.



In [ ]:
'''
tf.keras.preprocessing.text.one_hot(
    input_text,
    n,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    analyzer=None
)
'''

# Tokenize

In [24]:

from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
 'Good work',
 'Great effort',
 'nice work',
 'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)





1.   word_counts: A dictionary of words and their counts.
2.   word_docs: A dictionary of words and how many documents each appeared in.
1.   document_count:An integer count of the total number of documents that were used to fit the Tokenizer.
2.   word_index: A dictionary of words and their uniquely assigned integers.



In [25]:
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'well': 1, 'done': 1, 'work': 2, 'good': 1, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1})


The texts_to_matrix() function on the Tokenizer can be used to create one vector per document provided per input. The length of the vectors is the total size of the vocabulary.

This function provides a suite of standard bag-of-words model text encoding schemes that can be provided via a mode argument to the function.

The modes available include:

‘binary‘: Whether or not each word is present in the document. This is the default.
‘count‘: The count of each word in the document.
‘tfidf‘: The Text Frequency-Inverse DocumentFrequency (TF-IDF) scoring for each word in the document.
‘freq‘: The frequency of each word as a ratio of words within each document.

In [26]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
 'Good work',
 'Great effort',
 'nice work',
 'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'well': 1, 'done': 1, 'work': 2, 'good': 1, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1})
[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Running the example above fits the Tokenizer with 5 small documents. The details of the fit Tokenizer are printed. Then the 5 documents are encoded using a word count.

Each document is encoded as a 9-element vector with one position for each word and the chosen encoding scheme value for each word position. In this case, a simple word count mode is used.